**Summary**

This file serves to merge Binance Future trades that have the same date, pair and side into one trade and export it as a CSV file. If prices are different, ie. due to a market order, then the average price is calculated. The USDT prices are then converted to AUD using RBA daily exchange rates for 2018-current given by:
https://www.rba.gov.au/statistics/historical-data.html#exchange-rates

If the exchange rate does not exist (weekend or public holiday), the adjacent date is used instead.


**Requirements**
*   Must be in a CSV Format
*   Trades ordered by date
*   Columns should have the following labels: 
    
    `Date(UTC) || Pair || Side || Price || Quantity || Amount || Fee || Realized Profit`


In [ ]:
# Running this section is optional if you already have uploaded the data
# from google.colab import files
# futures_csv_file = files.upload()
# daily_rates = files.upload()

In [ ]:
import pandas as pd
import io
from datetime import datetime, timedelta
from dateutil import tz

# Rename this to whatever your trade file and forex data is called
trade_file_name = 'ivan_futures'
forex_history_file_name = '2018-current'
dotExtension = '.csv'

# Source to Series ID map
source_series_df = pd.read_csv('./' + forex_history_file_name + dotExtension, skiprows=(lambda x: x not in [8, 10]), header=None)
source_series_df.dropna(axis = 1, how ='all', inplace=True)
source_series_map = {}
for column, (source, series_id) in source_series_df.iteritems():
  if not (source == 'Source'):
    source_series_map[series_id] = source

# Exchange rate data for 2018-current
df_rates = pd.read_csv('./' + forex_history_file_name + dotExtension, skiprows=10)

# Future trade data
df_trades = pd.read_csv('./' + trade_file_name + dotExtension)
# df_trades.head()
df_rates.head()

,Series ID,FXRUSD,FXRTWI,FXRCR,FXRJY,FXREUR,FXRSKW,FXRUKPS,FXRSD,FXRIRE,FXRTB,FXRNZD,FXRNTD,FXRMR,FXRIR,FXRVD,FXRUAED,FXRPNGK,FXRHKD,FXRCD,FXRSARD,FXRSF,FXRPHP,FXRSDR,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67
0,02-Jan-2018,0.7837,65.1,5.0948,88.33,0.6522,831.98,0.5799,1.0449,49.92,25.47,1.1016,23.19,3.1599,10614.0,17798.0,2.8781,2.5321,6.1257,0.9824,NaN,0.7631,NaN,0.5503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,03-Jan-2018,0.7816,64.8,5.0842,87.80,0.6485,832.76,0.5747,1.0393,49.65,25.36,1.1021,23.11,3.1463,10551.0,17749.0,2.8704,2.5254,6.1104,0.9783,NaN,0.7597,NaN,0.5473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04-Jan-2018,0.7835,65.0,5.0983,88.27,0.6519,834.07,0.5795,1.0421,49.76,25.28,1.1029,23.19,3.1481,10541.0,17793.0,2.8774,2.5315,6.1260,0.9820,NaN,0.7660,NaN,0.5493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,05-Jan-2018,0.7857,65.0,5.0916,88.69,0.6508,834.37,0.5791,1.0423,49.77,25.27,1.0972,23.19,3.1381,10535.0,17843.0,2.8855,2.5386,6.1439,0.9809,NaN,0.7655,NaN,0.5503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,08-Jan-2018,0.7842,65.0,5.0893,88.78,0.6521,836.45,0.5784,1.0429,49.64,25.23,1.0931,23.12,3.1274,10516.0,17809.0,2.8800,2.5338,6.1335,0.9726,NaN,0.7652,NaN,0.5495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def add_to_final_trades(combined_row, df_final, source_series_map, series_id):
  """
  Add a new row to the final trades dataframe

  Convert USDT to AUD assuming that USDT is equal to USD. Then combine the new
  data into a row and append to the final trades dataframe. An adjacent date
  is used if the exchange rate does not exist for that day. ie. weekends or 
  public holidays

  :combined_row: list[str]
  
  Parameters
  ----------
  combined_row : df row
    Dataframe row containing the raw data to add to the final dataframe
  df_final : df
    Dataframe of final trades to be updated
  source_series_map : dict
    Dictionary mapping series ID to source
  series_id : str
    The series id to check for exchange rates

  Returns
  -------
  df
      A dataframe of the newly updated trades
  """
  # Recalcuate the average price
  combined_row['Price'] = float(combined_row['Amount'].split()[0].replace(',', '')) / combined_row['Quantity']
  
  # Convert USDT bases to AUD
  [combined_amount, combined_amount_quote] = combined_row['Amount'].split()
  [combined_fee, combined_fee_quote] = combined_row['Fee'].split()

  if combined_amount_quote == 'USDT' or combined_fee_quote == 'USDT':
    utc = datetime.strptime(combined_row['Date(UTC)'], '%d/%m/%Y  %H:%M')
    utc = utc.replace(tzinfo=from_zone)
    local_time = utc.astimezone(to_zone)
    local_date = local_time.strftime('%d-%b-%Y')
    df_selected_rate = df_rates.loc[df_rates['Series ID'] == local_date]

    # Check adjacent days if rate does not exist and use that instead
    if len(df_selected_rate) == 0:
      day_deltas = [-1, 1]
      for day_delta in day_deltas:
        new_date = local_time - timedelta(days=day_delta)
        new_local_date = new_date.strftime('%d-%b-%Y')
        df_selected_rate = df_rates.loc[df_rates['Series ID'] == new_local_date]
        if len(df_selected_rate) == 1:
          print('No Data', local_date, 'using', new_local_date, 'instead')
          local_date = new_local_date
          break

    if len(df_selected_rate) == 1:
      # The rate is stored as $1 AUD = usd_rate

      # Find a usd rate if it exists from the list of series ids
      usd_rate = None
      usd_rate = df_selected_rate.iloc[0][series_id]

      if usd_rate:
        if combined_amount_quote == 'USDT':
          combined_row['AUD/Amount Quote'] = usd_rate
          combined_row['AUD/Amount Source'] = source_series_map[series_id]
          combined_row['Amount(AUD)'] = float(combined_amount.replace(',', '')) / usd_rate

        if combined_fee_quote == 'USDT':
          combined_row['AUD/Fee Quote'] = usd_rate
          combined_row['AUD/Fee Source'] = source_series_map[series_id]
          combined_row['Fee(AUD)'] = float(combined_fee.replace(',', '')) / usd_rate

    else: 
      print('No Data', local_date)

  return df_final.append(combined_row, ignore_index=True)

In [ ]:
# Loop through the rows and if its the same as the previous one, combine them
# If its not the same, add the combined trade to a new dataframe

# Constants
from_zone = tz.gettz('UTC') # Timezone that the future trades are saved in
to_zone = tz.gettz('Australia/Melbourne') # Timezone to convert to
columns = ['Date(UTC)', 'Pair', 'Side', 'Price', 'Quantity', 'Amount', 'Fee', 'Realized Profit', 'AUD/Amount Quote', 'AUD/Fee Quote', 'AUD/Amount Source', 'AUD/Fee Source', 'Amount(AUD)', 'Fee(AUD)']

combined_row = df_trades.iloc[0].copy()
prev_row = df_trades.iloc[0].copy()
df_final = pd.DataFrame(columns=columns)
for current_row in df_trades.iloc[1:].iterrows():
  same_date = prev_row['Date(UTC)'] == current_row[1]['Date(UTC)']
  same_pair = prev_row['Pair'] == current_row[1]['Pair']
  same_side = prev_row['Side'] == current_row[1]['Side']

  if same_date and same_pair and same_side:
    # If previous constants are the same then combine the row
    combined_row['Quantity'] = combined_row['Quantity'] + current_row[1]['Quantity']
    combined_row['Realized Profit'] = combined_row['Realized Profit'] + current_row[1]['Realized Profit']
    
    # Check if the Amount and Fee are quoted in the same token
    # as the combined row
    [combined_amount, combined_amount_quote] = combined_row['Amount'].split()
    [current_amount, current_amount_quote] = current_row[1]['Amount'].split()
    [combined_fee, combined_fee_quote] = combined_row['Fee'].split()
    [current_fee, current_fee_quote] = current_row[1]['Fee'].split()

    if (combined_amount_quote == current_amount_quote) and (combined_fee_quote == current_fee_quote):
      combined_row['Amount'] = str(float(combined_amount.replace(',', '')) + float(current_amount.replace(',', ''))) + ' ' + combined_amount_quote
      combined_row['Fee'] = str(float(combined_fee.replace(',', '')) + float(current_fee.replace(',', ''))) + ' ' + combined_fee_quote
    else:
      raise Exception('Units are not the same in row ' + str(current_row[0]))
  else:
    # If not the same, then create a new combined trade and set the previous one
    df_final = add_to_final_trades(combined_row, df_final, source_series_map, "FXRUSD")
    combined_row = current_row[1].copy()
    
  prev_row = current_row[1]

# Add the last row to the final trades
df_final = add_to_final_trades(combined_row, df_final, source_series_map, "FXRUSD")
df_final.head()

No Data 09-May-2021 using 10-May-2021 instead
No Data 09-May-2021 using 10-May-2021 instead


,Date(UTC),Pair,Side,Price,Quantity,Amount,Fee,Realized Profit,AUD/Amount Quote,AUD/Fee Quote,AUD/Amount Source,AUD/Fee Source,Amount(AUD),Fee(AUD)
0,11/01/2021 10:34,ETHUSDT,SELL,1126.740000,0.719,810.1260600000 USDT,-0.3240504200 USDT,0.00000,0.7697,0.7697,WM/Reuters,WM/Reuters,1052.521840,-0.421009
1,11/01/2021 10:39,ETHUSDT,BUY,1118.690000,0.719,804.33811 USDT,-0.16086761 USDT,5.78795,0.7697,0.7697,WM/Reuters,WM/Reuters,1045.002092,-0.209000
2,11/01/2021 11:04,ETHUSDT,SELL,1093.940000,2.554,"2,793.9227600000 USDT",-1.1175691000 USDT,0.00000,0.7697,0.7697,WM/Reuters,WM/Reuters,3629.885358,-1.451954
3,11/01/2021 11:15,ETHUSDT,BUY,1073.690000,2.554,"2,742.2042600000 USDT",-0.5484408500 USDT,51.71850,0.7697,0.7697,WM/Reuters,WM/Reuters,3562.692296,-0.712538
4,11/01/2021 11:16,ETHUSDT,BUY,1080.318279,2.144,2316.2023900000004 USDT,-0.92648094 USDT,0.00000,0.7697,0.7697,WM/Reuters,WM/Reuters,3009.227478,-1.203691


In [ ]:
# Save file
df_final.to_csv(trade_file_name + '_combined' + dotExtension, index=False)

In [ ]:
#df_final